## Introduction to fixed / random effects 

We will need to know :
    - what is the variance of a linear combinaison of variables
    - play with the kroneker product
    - look at scipy.stat module

### First, checkout pna2015 and create your own day14 branch

### Variance of linear combinaison of (random) variables 

Let's $\textbf{y}$ for instance the value of a voxel in an fmri image) be our variable. If we repeat the experiment, we will observe $y_1, y_2, ..$. $\textbf{y}$ could also be the value of a contrast for a given subject, and we observe multiple subjects. 


Let's $\textbf{z}$ be another variable. 

Computing the variance (or, estimating the variance) of $\textbf{y}$ or $\textbf{z}$ necessitate that we have several observations.

The variance is defined as the true mean of the squared deviation of the observation to the true mean of the data. We need to estimate this. 

First, we estimate the mean : $\bar{y} = \frac{1}{n}\sum_i^n{y_i}$
Second we estimate the variation around this mean: 

$$ \widehat{\textrm{Var}}(\textbf{y}) = \frac{1}{n}\sum_i^n{(y_i - \bar{y})}^2$$

Notice again that this is just an estimation. The true variance is a fixed number, but we will get a different answer each time we have a different dataset. 

The covariance of two variables can be estimated with: 

$$ \widehat{\textrm{Cov}}(\textbf{y}, \textbf{z}) 
= \frac{1}{n}\sum_i^n{(y_i - \bar{y})(z_i - \bar{z})}$$


Notice that the covariance between $x$ and $x$ is the variance of $x$, and that the covariance is symetric: Cov(x,y) = Cov(y,x). Covariances are closely related to the Pearson correlation coefficient $c$, $c(x,y) = \frac{\textrm{Cov}(x,y)}{\sqrt(\textrm{Var}(x))\sqrt(\textrm{Var}(y))}$

Now, we can show that:

$$ \textrm{Cov}(\textbf{x} + \textbf{y}, \textbf{z}) = \textrm{Cov}(\textbf{x}, \textbf{z}) + \textrm{Cov}(\textbf{y}, \textbf{z})
$$

and that

$$ \textrm{Cov}(a\textbf{y}, \textbf{z}) = a\textrm{Cov}\textrm{Cov}(\textbf{y}, \textbf{z})
$$



$$ \textrm{Var}(\textbf{y} + \textbf{z}) = \textrm{Var}(\textbf{y}) + \textrm{Var}(\textbf{z}) + 2\textrm{Cov}(\textbf{y},\textbf{z})
$$

and 

$$ \textrm{Var}(a \, \textbf{y}) = a^2 \textrm{Var}(\textbf{y}) $$

Above, we droped the estimation sign (the hat) because these relations are true not only for our estimated values as defined above, but for the actual variances and covariances. You can check these relations easily with a given set of values taken by the variables.

Now this is the result that will be so useful in countless situations. Imaging that instead of having one variable $x$, we have a set of variables that we can represent by a vector of say, $p$ variables. Let's call this set of variable a vector $\overrightarrow{\bf{x}}$, each element of $\overrightarrow{\bf{x}}$ is one variable ${\bf{x_i}}$, and there are $p$ of those. Now, if I do my experiment again and again, say n times, I will observe n time the vector x, therefore n times a vector of p numbers. The variance-covariance of this vector $\overrightarrow{\bf{x}}$ is by definition the $p\times p$ matrix $\bf{V_x}$ with each element of $\bf{V_x}$ being:

$$ \bf{V_x(i,j)} = \textrm{Cov}(\bf{x_i}, \bf{x_j})$$


So, for instance, if the p variables are not correlated, the (actual) variance covariance matrix will be diagonal, all the off diagonal terms will be 0. The estimated one of course could and most likely will have non zero values off the diagonal. 

And there is the great thing. Imaging you are taking linear combinaisons of the $\bf{x_i}$. You therefore are creating new variables $\bf{y_i}$ that are just sums and scalings of the initial $\bf{x_i}$.  This can be represented as a multiplication by a matrix, say $\bf{A}$, with $\overrightarrow{\bf{y}} = \bf{A}\overrightarrow{\bf{x}}$. 

What is the variance covariance of $\overrightarrow{\bf{y}}$ ?  It is :

$$ \bf{V_y} = \bf{A} \bf{V_x} \bf{A}^T $$ 

If you take $p=2$, this is simple to check. Let's check that experiment agree with theory. 


In [1]:
# create a vector of 3 independant random variables 0,2
import numpy as np

x = np.random.normal(0,2, size=(3,1))

In [2]:
print(x)

[[-2.09072634]
 [-0.58351668]
 [-4.59530207]]


In [4]:
# now, let's mix them and create x1 - x2, x2 - x3, x1 + x2
import sympy as smp

A = np.asarray([[1, -1, 0], [0, 1, -1], [1, 1, 0]])
print(A)

#to do : make it lovely with sympy
#a = smp.Matrix(A)
#b = smp.Matrix(x)
#c = smp.Matrix(y)
#c = a*b


# our new y variables ys are 
y = A.dot(x)

print("\n y = A x : \n\n {} = \n {} {}".format(y,A,x))

[[ 1 -1  0]
 [ 0  1 -1]
 [ 1  1  0]]

 y = A x : 

 [[-1.50720966]
 [ 4.01178539]
 [-2.67424302]] = 
 [[ 1 -1  0]
 [ 0  1 -1]
 [ 1  1  0]] [[-2.09072634]
 [-0.58351668]
 [-4.59530207]]


In [5]:
# What is the variance covariance of x ?
# first, take many realizations:

n = 100000
x = np.random.normal(0,2, size=(3,n))

print("\n Estimation of the variance covariance of x: \n")

print(np.cov(x))

print("\n compare with the theoretical variance covariance of x:")

V_x = 4*np.eye(3)
print(V_x)

print("\n Close ! we get better results if n increases ...\n")

# What is the covariance of y ?
print("\n What is the estimated covariance of y ? \n")
      
y = A.dot(x)

print(np.cov(y))

# what is the theoretical covariance ? 
print("\n compare with the theoretical variance covariance of y : A V_x A.T:")

print(A.dot(V_x).dot(A.T))


 Estimation of the variance covariance of x: 

[[  4.01042282e+00   2.94963494e-03   1.27249148e-03]
 [  2.94963494e-03   3.99703007e+00  -1.05341850e-03]
 [  1.27249148e-03  -1.05341850e-03   4.00782911e+00]]

 compare with the theoretical variance covariance of x:
[[ 4.  0.  0.]
 [ 0.  4.  0.]
 [ 0.  0.  4.]]

 Close ! we get better results if n increases ...


 What is the estimated covariance of y ? 

[[ 8.00155363 -3.99640635  0.01339275]
 [-3.99640635  8.00696602  3.99976063]
 [ 0.01339275  3.99976063  8.01335217]]

 compare with the theoretical variance covariance of y : A V_x A.T:
[[ 8. -4.  0.]
 [-4.  8.  4.]
 [ 0.  4.  8.]]


###  The kroneker product : useful for creating array with patterns

In [6]:
one_matrix = np.asarray([[2, 3],[4,1]])
the_identity = np.eye(2)

In [7]:
K = np.kron(the_identity, one_matrix)

In [8]:
one_matrix

array([[2, 3],
       [4, 1]])

In [9]:
the_identity

array([[ 1.,  0.],
       [ 0.,  1.]])

In [10]:
K


array([[ 2.,  3.,  0.,  0.],
       [ 4.,  1.,  0.,  0.],
       [ 0.,  0.,  2.,  3.],
       [ 0.,  0.,  4.,  1.]])

In [11]:
c = np.array([1, -1, 0, 0 ])
o = np.ones((3))
k = np.kron(o, c)

In [12]:
k

array([ 1., -1.,  0.,  0.,  1., -1.,  0.,  0.,  1., -1.,  0.,  0.])

In [13]:
o

array([ 1.,  1.,  1.])